<a href="https://colab.research.google.com/github/Malhinova/Kaggle-Competition/blob/main/notebook4aa02ce91f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'tabular-playground-series-jul-2021:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F28007%2F2425957%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240423%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240423T134243Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D98f922d9e40245b0b02aa59045b84ed9ad2ffc1328945f2fc003ae03d471be3e85e69f4728813d0c0ba5b69daaa023d61cab6cd43b32603710d6fbb754253bb7844250e9efd7158bffce1b19cb7b510ea788ad8ff10ec288cfef06cf16887c78967cb74eb888f7e0cd2c6a3bbf12fa7d4d30dc16445b8b603400dfeb14539cbf519946b8c74f187fb23e9411094284f7f53ca7254cca71d53bd3b49c65b293d328920ca4925e93d0e4ec54db1ee14776685858394ff6e42a390e23701712e6074029149127a3ba54146fc7caa828b26cb560f5c1a9a8e6eddad5d18d20f8dac73d2c2b531028e4df4b2e244585a79a4167349faeaf10d3854c9eb3d94a045a94'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os, shutil

import torch
from torch import nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import random
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
data=pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
data.head()


In [ ]:
date_time_column = data['date_time']
targets=['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']
# for row in targets:
#     plt.figure(figsize=(8, 6))
#     sns.scatterplot(y= data[row],x=date_time_column )
#     plt.show()

In [ ]:
data_copy=data.copy()
data_copy['date_time'] = pd.to_datetime(data_copy['date_time'])
data_copy['month'] = data_copy['date_time'].dt.month
data_copy['day'] = data_copy['date_time'].dt.day
data_copy['hour'] = data_copy['date_time'].dt.hour
data_copy['day_week'] = data_copy['date_time'].dt.day_of_week



In [ ]:
def find_hour(hour):
    if (hour>=0 and hour<6):
        return 1 #Night
    if hour>=6 and hour<12:
        return 2 #Morning
    if hour>=12 and hour<18:
        return 3 #Day
    if hour>=18 and hour<24:
        return 4 #Evening

In [ ]:
hour_list=[]
for hour in data_copy['hour']:
    part_day = find_hour(hour)
    hour_list.append(part_day)
data_copy['part_day']=hour_list
data_copy.head()

In [ ]:
columns = list(data_copy.columns)

for col in ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']:
    columns.remove(col)
columns += ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']

# Rearrange the columns in the final_data DataFrame
data_copy = data_copy[columns]
data_copy.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features_n =['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']
final_data=data_copy.copy()
final_data[features_n]  = scaler.fit_transform(final_data[features_n])
final_data.head()

In [ ]:
numeric_columns_test = final_data.select_dtypes(include=np.number).columns

correlation_matrix = final_data[numeric_columns_test].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
features_1 =['sensor_1', 'sensor_2','sensor_5', 'part_day']
features_2 =['sensor_2','sensor_5','part_day']
features_3 =['deg_C', 'relative_humidity', 'sensor_2', 'sensor_3', 'sensor_4','sensor_5']
X1=final_data[features_1]
X2=final_data[features_2]
X3=final_data[features_3]

Y = final_data[targets]
Y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X1, Y[targets[0]])


In [ ]:
from xgboost import XGBRegressor

xgb_regressor1 = XGBRegressor(n_estimators=400, learning_rate=0.05)
xgb_regressor1.fit(X_train, y_train,
             early_stopping_rounds=5,
             eval_set=[(X_valid, y_valid)],
             verbose=False)

In [ ]:
from sklearn.metrics import mean_absolute_error

predictions = xgb_regressor1.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X2, Y[targets[1]])

In [ ]:
xgb_regressor2 = XGBRegressor(n_estimators=500, learning_rate=0.01)
xgb_regressor2.fit(X_train, y_train,
             early_stopping_rounds=4,
             eval_set=[(X_valid, y_valid)],
             verbose=False)
predictions = xgb_regressor2.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X3, Y[targets[2]])

In [ ]:
xgb_regressor3 = XGBRegressor(n_estimators=500, learning_rate=0.05)
xgb_regressor3.fit(X_train, y_train,
             early_stopping_rounds=5,
             eval_set=[(X_valid, y_valid)],
             verbose=False)
predictions = xgb_regressor3.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

In [ ]:
test_data=pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')
test_data.head()
date_time_column2 = test_data['date_time']

In [ ]:
test_data['date_time'] = pd.to_datetime(test_data['date_time'])
test_data['month'] = test_data['date_time'].dt.month
test_data['day'] = test_data['date_time'].dt.day
test_data['hour'] = test_data['date_time'].dt.hour
test_data['day_week'] = test_data['date_time'].dt.day_of_week
test_data.head()

In [ ]:
hour_list=[]
for hour in test_data['hour']:
    part_day = find_hour(hour)
    hour_list.append(part_day)
test_data['part_day']=hour_list
test_data.head()

In [ ]:
features_n =['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']
test_data[features_n]  = scaler.fit_transform(test_data[features_n])
test_data.head()

In [ ]:

X1=test_data[features_1]
X2=test_data[features_2]
X3=test_data[features_3]
X1.head()

In [ ]:
out1=xgb_regressor1.predict(X1)
out2=xgb_regressor2.predict(X2)
out3=xgb_regressor3.predict(X3)
output_df = pd.DataFrame({
    'date_time': date_time_column2,
    'target_carbon_monoxide': out1,
    'target_benzene': out2,
    'target_nitrogen_oxides': out3
})

output_df.head()


In [ ]:
# import numpy as np
# import xgboost as xgb

# # Define the parameters for XGBoost models
# xgb_params = {
#     'objective': 'reg:squarederror',  # Use regression objective function
#     'eval_metric': 'rmse',  # Root Mean Squared Error as evaluation metric
#     'max_depth': 6,  # Maximum depth of a tree
#     'learning_rate': 0.1,  # Learning rate
#     'n_estimators': 1000,  # Number of boosting rounds
#     'subsample': 0.8,  # Subsample ratio of the training instances
#     'colsample_bytree': 0.8,  # Subsample ratio of columns when constructing each tree
#     'random_state': None  # Different seed for each model
# }

# # Initialize a list to store XGBoost models
# xgb_models = []

# # Create 3 XGBoost models, one for each target
# for target in targets:
#     # Initialize XGBoost regressor
#     xgb_regressor = xgb.XGBRegressor(**xgb_params)
#     # Train the XGBoost model on the training data for the current target
#     xgb_regressor.fit(X_train, y_train[target])
#     # Append the trained model to the list
#     xgb_models.append(xgb_regressor)

# # Predictions from each XGBoost model
# all_predictions = np.zeros((len(X_test), len(targets)))

# # Make predictions using each XGBoost model and store them
# for i, model in enumerate(xgb_models):
#     predictions = model.predict(X_test)
#     all_predictions[:, i] = predictions

# # Compute RMSE for each target separately
# rmse = np.sqrt(np.mean((y_test - all_predictions) ** 2, axis=0))
# for i, target in enumerate(targets):
#     print(f"RMSE for {target}: {rmse[i]}")

In [ ]:
output_df.shape

In [ ]:
output_df.to_csv('submission.csv', index=False)

In [ ]:
# predicted_df.shape

In [ ]:
# predicted_df.to_csv('submission.csv', index=False)